<a href="https://colab.research.google.com/github/yash-manchanda/Animal-state-prediction/blob/master/Predictive_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/yash-manchanda/data

fatal: destination path 'data' already exists and is not an empty directory.


### Loadind CSV

In [0]:
import pandas as pd

In [0]:
train = pd.read_csv("/content/data/train.csv")
test = pd.read_csv("/content/data/test.csv")

In [4]:
train.head(5)

,animal_id_outcome,dob_year,dob_month,age_upon_intake,animal_type,breed,color,intake_condition,intake_type,sex_upon_intake,count,age_upon_intake_(days),age_upon_intake_(years),age_upon_intake_age_group,intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days,age_upon_outcome,date_of_birth,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,outcome_month,outcome_year,outcome_monthyear,outcome_weekday,outcome_hour,outcome_number,outcome_type
0,A006100,2007,7,10 years,Dog,Spinone Italiano Mix,Yellow/White,Normal,Stray,Neutered Male,1,3650,10.0,"(7.5, 10.0]",2017-12-07 00:00:00,12,2017,2017-12,Thursday,14,1.0,0 days 14:07:00.000000000,0.588194,10 years,2007-07-09 00:00:00,Neutered Male,3650,10.0,"(7.5, 10.0]",2017-12-07 14:07:00,12,2017,2017-12,Thursday,0,1.0,Return to Owner
1,A006100,2007,7,7 years,Dog,Spinone Italiano Mix,Yellow/White,Normal,Public Assist,Neutered Male,1,2555,7.0,"(5.0, 7.5]",2014-12-19 10:21:00,12,2014,2014-12,Friday,10,2.0,1 days 06:14:00.000000000,1.259722,7 years,2007-07-09 00:00:00,Neutered Male,2555,7.0,"(5.0, 7.5]",2014-12-20 16:35:00,12,2014,2014-12,Saturday,16,2.0,Return to Owner
2,A006100,2007,7,6 years,Dog,Spinone Italiano Mix,Yellow/White,Normal,Public Assist,Neutered Male,1,2190,6.0,"(5.0, 7.5]",2014-03-07 14:26:00,3,2014,2014-03,Friday,14,3.0,1 days 02:44:00.000000000,1.113889,6 years,2007-07-09 00:00:00,Neutered Male,2190,6.0,"(5.0, 7.5]",NaN,3,2014,2014-03,Saturday,17,3.0,Return to Owner
3,A047759,2004,4,10 years,Dog,Dachshund,Tricolor,Normal,Owner Surrender,Neutered Male,1,3650,10.0,"(7.5, 10.0]",2014-04-02 15:55:00,4,2014,2014-04,Wednesday,15,1.0,4 days 23:17:00.000000000,4.970139,10 years,2004-04-02 00:00:00,Neutered Male,3650,10.0,"(7.5, 10.0]",2014-04-07 15:12:00,4,2014,2014-04,Monday,15,1.0,Missing
4,A134067,1997,10,16 years,Dog,Shetland Sheepdog,Brown/White,Injured,Public Assist,Neutered Male,1,5840,16.0,"(15.0, 17.5]",2013-11-16 09:02:00,11,2013,2013-11,Saturday,9,1.0,0 days 02:52:00.000000000,0.119444,16 years,1997-10-16 00:00:00,Neutered Male,5840,16.0,"(15.0, 17.5]",2013-11-16 11:54:00,11,2013,2013-11,Saturday,11,1.0,Return to Owner


In [5]:
test.head(5)

,animal_id_outcome,dob_year,dob_month,age_upon_intake,animal_type,breed,color,intake_condition,intake_type,sex_upon_intake,count,age_upon_intake_(days),age_upon_intake_(years),age_upon_intake_age_group,intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days,age_upon_outcome,date_of_birth,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,outcome_month,outcome_year,outcome_monthyear,outcome_weekday,outcome_hour,outcome_number
0,A723133,2013,3,3 years,Cat,Domestic Shorthair Mix,Brown Tabby,Nursing,Stray,Intact Female,1,1095,3.000000,"(2.5, 5.0]",27-03-2016 11:31,3,2016,2016-03,Sunday,11,1,5 days 04:48:00.000000000,5.200000,3 years,27-03-2013 00:00,Spayed Female,1095,3.000000,"(2.5, 5.0]",01-04-2016 16:19,4,2016,2016-04,Friday,16,1
1,A723134,2016,2,4 weeks,Cat,Domestic Shorthair Mix,Brown Tabby,Nursing,Stray,Intact Female,1,28,0.076712,"(-0.025, 2.5]",27-03-2016 11:31,3,2016,2016-03,Sunday,11,1,34 days 06:11:00.000000000,34.257639,2 months,27-02-2016 00:00,Spayed Female,60,0.164384,"(-0.025, 2.5]",30-04-2016 17:42,4,2016,2016-04,Saturday,17,1
2,A723135,2016,2,4 weeks,Cat,Domestic Shorthair Mix,Brown Tabby,Nursing,Stray,Intact Male,1,28,0.076712,"(-0.025, 2.5]",27-03-2016 11:31,3,2016,2016-03,Sunday,11,1,32 days 12:29:00.000000000,32.520139,2 months,27-02-2016 00:00,Neutered Male,60,0.164384,"(-0.025, 2.5]",29-04-2016 00:00,4,2016,2016-04,Friday,0,1
3,A723136,2016,2,4 weeks,Cat,Domestic Shorthair Mix,Brown Tabby,Nursing,Stray,Intact Female,1,28,0.076712,"(-0.025, 2.5]",27-03-2016 11:31,3,2016,2016-03,Sunday,11,1,31 days 02:16:00.000000000,31.094444,1 month,27-02-2016 00:00,Intact Female,30,0.082192,"(-0.025, 2.5]",27-04-2016 13:47,4,2016,2016-04,Wednesday,13,1
4,A723137,2016,2,4 weeks,Cat,Domestic Shorthair Mix,Blue Tabby,Nursing,Stray,Intact Female,1,28,0.076712,"(-0.025, 2.5]",27-03-2016 11:31,3,2016,2016-03,Sunday,11,1,37 days 00:11:00.000000000,37.007639,2 months,27-02-2016 00:00,Intact Female,60,0.164384,"(-0.025, 2.5]",03-05-2016 11:42,5,2016,2016-05,Tuesday,11,1


### Feature Engineering

In [0]:
def new_animal_type(animal_type):            
  if "bird" in animal_type.lower():
    return "Other"
  elif "other" in animal_type.lower():
    return "Other"
  elif "dog" in animal_type.lower():
    return "Dog"
  elif "cat" in animal_type.lower():
    return "Cat"


train['new_animal_type'] = train.animal_type.astype(str).apply(new_animal_type)
test['new_animal_type'] = test.animal_type.astype(str).apply(new_animal_type)

In [0]:
def gender(Sex):
    if "unknown" in Sex.lower():
        return "Unknown"
    elif "female" in Sex.lower():
        return "Female"
    elif "male" in Sex.lower():
        return "Male"


train['gender'] = train.sex_upon_intake.astype(str).apply(gender)
test['gender'] = test.sex_upon_intake.astype(str).apply(gender)

In [0]:
def breed_type(breed):
    if "mix" in breed.lower():
      return "Mixed Breed"
    elif "/" in breed.lower():
      return "Single Unknown Breed"
    else:
      return "Single Breed"


train['new_breed'] = train.breed.astype(str).apply(breed_type)
test['new_breed'] = test.breed.astype(str).apply(breed_type)

In [0]:
def color_new(data):
  if "tricolor/" in data.lower():
    return "Quadcolor"
  elif "/tricolor" in data.lower():
    return "Quadcolor"
  elif "/" in data.lower():
    return "Bicolor"
  elif "tricolor" in data.lower():
    return "Tricolor"
  else:
    return "Single Color"


train['color_new'] = train.color.astype(str).apply(color_new)
test['color_new'] = test.color.astype(str).apply(color_new)

In [0]:
def extract_datetime(dataset):
  dataset["intake_date"] = pd.DatetimeIndex(dataset['intake_datetime']).day
  return dataset

train = extract_datetime(train)
test = extract_datetime(test)

### Treating Missing Values

In [11]:
train.isnull().sum()

animal_id_outcome                0
dob_year                         0
dob_month                        0
age_upon_intake                  0
animal_type                      0
breed                            0
color                            0
intake_condition                 0
intake_type                      0
sex_upon_intake                  1
count                            0
age_upon_intake_(days)           0
age_upon_intake_(years)          0
age_upon_intake_age_group        0
intake_datetime                  0
intake_month                     0
intake_year                      0
intake_monthyear                 0
intake_weekday                   0
intake_hour                      0
intake_number                    0
time_in_shelter                  0
time_in_shelter_days             0
age_upon_outcome                 0
date_of_birth                    0
sex_upon_outcome                 1
age_upon_outcome_(days)          0
age_upon_outcome_(years)         0
age_upon_outcome_age

In [12]:
test.isnull().sum()

animal_id_outcome             0
dob_year                      0
dob_month                     0
age_upon_intake               0
animal_type                   0
breed                         0
color                         0
intake_condition              0
intake_type                   0
sex_upon_intake               0
count                         0
age_upon_intake_(days)        0
age_upon_intake_(years)       0
age_upon_intake_age_group     0
intake_datetime               0
intake_month                  0
intake_year                   0
intake_monthyear              0
intake_weekday                0
intake_hour                   0
intake_number                 0
time_in_shelter               0
time_in_shelter_days          0
age_upon_outcome              0
date_of_birth                 0
sex_upon_outcome              0
age_upon_outcome_(days)       0
age_upon_outcome_(years)      0
age_upon_outcome_age_group    0
outcome_datetime              0
outcome_month                 0
outcome_

In [0]:
train = train.dropna(subset = ["gender"])            # dropping one missing value of gender from train dataframe as all the values are extracted from it

In [0]:
train = train.drop(['outcome_datetime'], axis = 1)              # dropped for now because of (NULL )
test = test.drop(['outcome_datetime'], axis = 1)

In [15]:
train

,animal_id_outcome,dob_year,dob_month,age_upon_intake,animal_type,breed,color,intake_condition,intake_type,sex_upon_intake,count,age_upon_intake_(days),age_upon_intake_(years),age_upon_intake_age_group,intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days,age_upon_outcome,date_of_birth,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_month,outcome_year,outcome_monthyear,outcome_weekday,outcome_hour,outcome_number,outcome_type,new_animal_type,gender,new_breed,color_new,intake_date
0,A006100,2007,7,10 years,Dog,Spinone Italiano Mix,Yellow/White,Normal,Stray,Neutered Male,1,3650,10.0,"(7.5, 10.0]",2017-12-07 00:00:00,12,2017,2017-12,Thursday,14,1.0,0 days 14:07:00.000000000,0.588194,10 years,2007-07-09 00:00:00,Neutered Male,3650,10.0,"(7.5, 10.0]",12,2017,2017-12,Thursday,0,1.0,Return to Owner,Dog,Male,Mixed Breed,Bicolor,7
1,A006100,2007,7,7 years,Dog,Spinone Italiano Mix,Yellow/White,Normal,Public Assist,Neutered Male,1,2555,7.0,"(5.0, 7.5]",2014-12-19 10:21:00,12,2014,2014-12,Friday,10,2.0,1 days 06:14:00.000000000,1.259722,7 years,2007-07-09 00:00:00,Neutered Male,2555,7.0,"(5.0, 7.5]",12,2014,2014-12,Saturday,16,2.0,Return to Owner,Dog,Male,Mixed Breed,Bicolor,19
2,A006100,2007,7,6 years,Dog,Spinone Italiano Mix,Yellow/White,Normal,Public Assist,Neutered Male,1,2190,6.0,"(5.0, 7.5]",2014-03-07 14:26:00,3,2014,2014-03,Friday,14,3.0,1 days 02:44:00.000000000,1.113889,6 years,2007-07-09 00:00:00,Neutered Male,2190,6.0,"(5.0, 7.5]",3,2014,2014-03,Saturday,17,3.0,Return to Owner,Dog,Male,Mixed Breed,Bicolor,7
3,A047759,2004,4,10 years,Dog,Dachshund,Tricolor,Normal,Owner Surrender,Neutered Male,1,3650,10.0,"(7.5, 10.0]",2014-04-02 15:55:00,4,2014,2014-04,Wednesday,15,1.0,4 days 23:17:00.000000000,4.970139,10 years,2004-04-02 00:00:00,Neutered Male,3650,10.0,"(7.5, 10.0]",4,2014,2014-04,Monday,15,1.0,Missing,Dog,Male,Single Breed,Tricolor,2
4,A134067,1997,10,16 years,Dog,Shetland Sheepdog,Brown/White,Injured,Public Assist,Neutered Male,1,5840,16.0,"(15.0, 17.5]",2013-11-16 09:02:00,11,2013,2013-11,Saturday,9,1.0,0 days 02:52:00.000000000,0.119444,16 years,1997-10-16 00:00:00,Neutered Male,5840,16.0,"(15.0, 17.5]",11,2013,2013-11,Saturday,11,1.0,Return to Owner,Dog,Male,Single Breed,Bicolor,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47798,A723128,2007,9,8 years,Cat,Domestic Longhair Mix,Black/White,Normal,Stray,Neutered Male,1,2920,8.0,"(7.5, 10.0]",2016-03-27 11:24:00,3,2016,2016-03,Sunday,11,1.0,21 days 12:36:00.000000000,21.525000,8 years,2007-09-27 00:00:00,Neutered Male,2920,8.0,"(7.5, 10.0]",4,2016,2016-04,Monday,0,1.0,Rto-Adopt,Cat,Male,Mixed Breed,Bicolor,27
47799,A723129,2015,3,1 year,Dog,Cairn Terrier Mix,Gray/White,Normal,Owner Surrender,Intact Male,1,365,1.0,"(-0.025, 2.5]",2016-03-27 11:44:00,3,2016,2016-03,Sunday,11,1.0,2 days 07:22:00.000000000,2.306944,1 year,2015-03-27 00:00:00,Neutered Male,365,1.0,"(-0.025, 2.5]",3,2016,2016-03,Tuesday,19,1.0,Adoption,Dog,Male,Mixed Breed,Bicolor,27
47800,A723130,2005,3,11 years,Dog,Shetland Sheepdog Mix,Brown/Tan,Normal,Stray,Spayed Female,1,4015,11.0,"(10.0, 12.5]",2016-03-27 11:44:00,3,2016,2016-03,Sunday,11,1.0,0 days 05:23:00.000000000,0.224306,11 years,2005-03-27 00:00:00,Spayed Female,4015,11.0,"(10.0, 12.5]",3,2016,2016-03,Sunday,17,1.0,Return to Owner,Dog,Female,Mixed Breed,Bicolor,27
47801,A723131,2011,3,5 years,Dog,Boxer/Catahoula,Brown Brindle/White,Normal,Stray,Intact Male,1,1825,5.0,"(2.5, 5.0]",2016-03-27 11:57:00,3,2016,2016-03,Sunday,11,1.0,13 days 01:40:00.000000000,13.069444,5 years,2011-03-27 00:00:00,Neutered Male,1825,5.0,"(2.5, 5.0]",4,2016,2016-04,Saturday,13,1.0,Adoption,Dog,Male,Single Unknown Breed,Bicolor,27


### Converting categorical string data into numerical by using label encoder

In [0]:
from sklearn import preprocessing
label_enc = preprocessing.LabelEncoder()

In [17]:
train.new_animal_type.unique()

array(['Dog', 'Cat', 'Other'], dtype=object)

In [18]:
train.intake_condition.unique()

array(['Normal', 'Injured', 'Aged', 'Sick', 'Other', 'Feral', 'Pregnant',
       'Nursing'], dtype=object)

In [19]:
train.intake_type.unique()

array(['Stray', 'Public Assist', 'Owner Surrender', 'Euthanasia Request',
       'Wildlife'], dtype=object)

In [20]:
train.gender.unique()

array(['Male', 'Female', 'Unknown'], dtype=object)

In [21]:
train.age_upon_intake_age_group.unique()

array(['(7.5, 10.0]', '(5.0, 7.5]', '(15.0, 17.5]', '(12.5, 15.0]',
       '(17.5, 20.0]', '(10.0, 12.5]', '(2.5, 5.0]', '(-0.025, 2.5]',
       '(20.0, 22.5]'], dtype=object)

In [22]:
train.intake_weekday.unique()

array(['Thursday', 'Friday', 'Wednesday', 'Saturday', 'Monday', 'Tuesday',
       'Sunday'], dtype=object)

In [23]:
train.new_breed.unique()

array(['Mixed Breed', 'Single Breed', 'Single Unknown Breed'],
      dtype=object)

In [24]:
train.color_new.unique()

array(['Bicolor', 'Tricolor', 'Single Color', 'Quadcolor'], dtype=object)

In [25]:
train.outcome_type.unique()

array(['Return to Owner', 'Missing', 'Transfer', 'Adoption', 'Euthanasia',
       'Rto-Adopt', 'Relocate', 'Died', 'Disposal'], dtype=object)

In [0]:
train["new_animal_type"] = label_enc.fit_transform(train["new_animal_type"])
test["new_animal_type"] = label_enc.fit_transform(test["new_animal_type"])

train["intake_condition"] = label_enc.fit_transform(train["intake_condition"])
test["intake_condition"] = label_enc.fit_transform(test["intake_condition"])

train["intake_type"] = label_enc.fit_transform(train["intake_type"])
test["intake_type"] = label_enc.fit_transform(test["intake_type"])

train["gender"] = label_enc.fit_transform(train["gender"])
test["gender"] = label_enc.fit_transform(test["gender"])

train["age_upon_intake_age_group"] = label_enc.fit_transform(train["age_upon_intake_age_group"])
test["age_upon_intake_age_group"] = label_enc.fit_transform(test["age_upon_intake_age_group"])

train["age_upon_outcome_age_group"] = label_enc.fit_transform(train["age_upon_outcome_age_group"])
test["age_upon_outcome_age_group"] = label_enc.fit_transform(test["age_upon_outcome_age_group"])

train["intake_weekday"] = label_enc.fit_transform(train["intake_weekday"])
test["intake_weekday"] = label_enc.fit_transform(test["intake_weekday"])

train["outcome_weekday"] = label_enc.fit_transform(train["outcome_weekday"])
test["outcome_weekday"] = label_enc.fit_transform(test["outcome_weekday"])

train["new_breed"] = label_enc.fit_transform(train["new_breed"])
test["new_breed"] = label_enc.fit_transform(test["new_breed"])

train["color_new"] = label_enc.fit_transform(train["color_new"])
test["color_new"] = label_enc.fit_transform(test["color_new"])

In [27]:
train.new_animal_type.unique()

array([1, 0, 2])

In [28]:
train.intake_condition.unique()

array([3, 2, 0, 7, 5, 1, 6, 4])

In [29]:
train.intake_type.unique()

array([3, 2, 1, 0, 4])

In [30]:
train.gender.unique()

array([1, 0, 2])

In [31]:
train.age_upon_intake_age_group.unique()

array([8, 7, 3, 2, 4, 1, 5, 0, 6])

In [32]:
train.intake_weekday.unique()

array([4, 0, 6, 2, 1, 5, 3])

In [33]:
train.age_upon_outcome_age_group.unique()

array([8, 7, 3, 2, 4, 1, 5, 0, 6])

In [34]:
train.outcome_weekday.unique()

array([4, 2, 1, 3, 0, 5, 6])

In [35]:
train.new_breed.unique()

array([0, 1, 2])

In [36]:
train.outcome_type.unique()

array(['Return to Owner', 'Missing', 'Transfer', 'Adoption', 'Euthanasia',
       'Rto-Adopt', 'Relocate', 'Died', 'Disposal'], dtype=object)

### Feature Engineering of the modified features

In [0]:
def age_group_diff(data):
  diff = abs(data['age_upon_outcome_age_group'] - data['age_upon_intake_age_group'])
  age_diff = []
  for i in diff:
    if i == 0:
      age_diff.append(0)
    else:
      age_diff.append(1)
  return age_diff


train['age_group_difference'] = age_group_diff(train)
test['age_group_difference'] = age_group_diff(test)

In [0]:
def year_in_awc(data):
  diff = round(abs(data['age_upon_intake_(years)']	- data['age_upon_outcome_(years)']))
  return diff


train['years_in_awc'] = year_in_awc(train)
test['years_in_awc'] = year_in_awc(test)

In [0]:
def months_in_awc(data):
  if (data['outcome_month']> data['intake_month']):
    diff = round(abs((12 - data['outcome_month'])	+ data['intake_month']))
    return diff
  else:
    diff = round(abs(data['outcome_month'] - data['intake_month']))
    return diff


train['months_in_awc'] = year_in_awc(train)
test['months_in_awc'] = year_in_awc(test)

In [0]:
train['time_in_shelter_days'] = round(train['time_in_shelter_days'])
test['time_in_shelter_days'] = round(test['time_in_shelter_days'])

### Dropping unwanted or repeated coulumns

In [41]:
train['dob_month'].isin(train['outcome_month']).value_counts()   

True    47802
Name: dob_month, dtype: int64

In [42]:
train

,animal_id_outcome,dob_year,dob_month,age_upon_intake,animal_type,breed,color,intake_condition,intake_type,sex_upon_intake,count,age_upon_intake_(days),age_upon_intake_(years),age_upon_intake_age_group,intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days,age_upon_outcome,date_of_birth,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_month,outcome_year,outcome_monthyear,outcome_weekday,outcome_hour,outcome_number,outcome_type,new_animal_type,gender,new_breed,color_new,intake_date,age_group_difference,years_in_awc,months_in_awc
0,A006100,2007,7,10 years,Dog,Spinone Italiano Mix,Yellow/White,3,3,Neutered Male,1,3650,10.0,8,2017-12-07 00:00:00,12,2017,2017-12,4,14,1.0,0 days 14:07:00.000000000,1.0,10 years,2007-07-09 00:00:00,Neutered Male,3650,10.0,8,12,2017,2017-12,4,0,1.0,Return to Owner,1,1,0,0,7,0,0.0,0.0
1,A006100,2007,7,7 years,Dog,Spinone Italiano Mix,Yellow/White,3,2,Neutered Male,1,2555,7.0,7,2014-12-19 10:21:00,12,2014,2014-12,0,10,2.0,1 days 06:14:00.000000000,1.0,7 years,2007-07-09 00:00:00,Neutered Male,2555,7.0,7,12,2014,2014-12,2,16,2.0,Return to Owner,1,1,0,0,19,0,0.0,0.0
2,A006100,2007,7,6 years,Dog,Spinone Italiano Mix,Yellow/White,3,2,Neutered Male,1,2190,6.0,7,2014-03-07 14:26:00,3,2014,2014-03,0,14,3.0,1 days 02:44:00.000000000,1.0,6 years,2007-07-09 00:00:00,Neutered Male,2190,6.0,7,3,2014,2014-03,2,17,3.0,Return to Owner,1,1,0,0,7,0,0.0,0.0
3,A047759,2004,4,10 years,Dog,Dachshund,Tricolor,3,1,Neutered Male,1,3650,10.0,8,2014-04-02 15:55:00,4,2014,2014-04,6,15,1.0,4 days 23:17:00.000000000,5.0,10 years,2004-04-02 00:00:00,Neutered Male,3650,10.0,8,4,2014,2014-04,1,15,1.0,Missing,1,1,1,3,2,0,0.0,0.0
4,A134067,1997,10,16 years,Dog,Shetland Sheepdog,Brown/White,2,2,Neutered Male,1,5840,16.0,3,2013-11-16 09:02:00,11,2013,2013-11,2,9,1.0,0 days 02:52:00.000000000,0.0,16 years,1997-10-16 00:00:00,Neutered Male,5840,16.0,3,11,2013,2013-11,2,11,1.0,Return to Owner,1,1,1,0,16,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47798,A723128,2007,9,8 years,Cat,Domestic Longhair Mix,Black/White,3,3,Neutered Male,1,2920,8.0,8,2016-03-27 11:24:00,3,2016,2016-03,3,11,1.0,21 days 12:36:00.000000000,22.0,8 years,2007-09-27 00:00:00,Neutered Male,2920,8.0,8,4,2016,2016-04,1,0,1.0,Rto-Adopt,0,1,0,0,27,0,0.0,0.0
47799,A723129,2015,3,1 year,Dog,Cairn Terrier Mix,Gray/White,3,1,Intact Male,1,365,1.0,0,2016-03-27 11:44:00,3,2016,2016-03,3,11,1.0,2 days 07:22:00.000000000,2.0,1 year,2015-03-27 00:00:00,Neutered Male,365,1.0,0,3,2016,2016-03,5,19,1.0,Adoption,1,1,0,0,27,0,0.0,0.0
47800,A723130,2005,3,11 years,Dog,Shetland Sheepdog Mix,Brown/Tan,3,3,Spayed Female,1,4015,11.0,1,2016-03-27 11:44:00,3,2016,2016-03,3,11,1.0,0 days 05:23:00.000000000,0.0,11 years,2005-03-27 00:00:00,Spayed Female,4015,11.0,1,3,2016,2016-03,3,17,1.0,Return to Owner,1,0,0,0,27,0,0.0,0.0
47801,A723131,2011,3,5 years,Dog,Boxer/Catahoula,Brown Brindle/White,3,3,Intact Male,1,1825,5.0,5,2016-03-27 11:57:00,3,2016,2016-03,3,11,1.0,13 days 01:40:00.000000000,13.0,5 years,2011-03-27 00:00:00,Neutered Male,1825,5.0,5,4,2016,2016-04,2,13,1.0,Adoption,1,1,2,0,27,0,0.0,0.0


In [43]:
test

,animal_id_outcome,dob_year,dob_month,age_upon_intake,animal_type,breed,color,intake_condition,intake_type,sex_upon_intake,count,age_upon_intake_(days),age_upon_intake_(years),age_upon_intake_age_group,intake_datetime,intake_month,intake_year,intake_monthyear,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days,age_upon_outcome,date_of_birth,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_month,outcome_year,outcome_monthyear,outcome_weekday,outcome_hour,outcome_number,new_animal_type,gender,new_breed,color_new,intake_date,age_group_difference,years_in_awc,months_in_awc
0,A723133,2013,3,3 years,Cat,Domestic Shorthair Mix,Brown Tabby,4,3,Intact Female,1,1095,3.000000,5,27-03-2016 11:31,3,2016,2016-03,3,11,1,5 days 04:48:00.000000000,5.0,3 years,27-03-2013 00:00,Spayed Female,1095,3.000000,5,4,2016,2016-04,0,16,1,0,0,0,2,27,0,0.0,0.0
1,A723134,2016,2,4 weeks,Cat,Domestic Shorthair Mix,Brown Tabby,4,3,Intact Female,1,28,0.076712,0,27-03-2016 11:31,3,2016,2016-03,3,11,1,34 days 06:11:00.000000000,34.0,2 months,27-02-2016 00:00,Spayed Female,60,0.164384,0,4,2016,2016-04,2,17,1,0,0,0,2,27,0,0.0,0.0
2,A723135,2016,2,4 weeks,Cat,Domestic Shorthair Mix,Brown Tabby,4,3,Intact Male,1,28,0.076712,0,27-03-2016 11:31,3,2016,2016-03,3,11,1,32 days 12:29:00.000000000,33.0,2 months,27-02-2016 00:00,Neutered Male,60,0.164384,0,4,2016,2016-04,0,0,1,0,1,0,2,27,0,0.0,0.0
3,A723136,2016,2,4 weeks,Cat,Domestic Shorthair Mix,Brown Tabby,4,3,Intact Female,1,28,0.076712,0,27-03-2016 11:31,3,2016,2016-03,3,11,1,31 days 02:16:00.000000000,31.0,1 month,27-02-2016 00:00,Intact Female,30,0.082192,0,4,2016,2016-04,6,13,1,0,0,0,2,27,0,0.0,0.0
4,A723137,2016,2,4 weeks,Cat,Domestic Shorthair Mix,Blue Tabby,4,3,Intact Female,1,28,0.076712,0,27-03-2016 11:31,3,2016,2016-03,3,11,1,37 days 00:11:00.000000000,37.0,2 months,27-02-2016 00:00,Intact Female,60,0.164384,0,5,2016,2016-05,5,11,1,0,0,0,2,27,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31864,A769060,2018,3,2 weeks,Cat,Domestic Shorthair Mix,Tortie,3,3,Unknown,1,14,0.038356,0,29-03-2018 16:23,3,2018,2018-03,4,16,1,0 days 01:51:00.000000000,0.0,2 weeks,14-03-2018 00:00,Unknown,14,0.038356,0,3,2018,2018-03,4,18,1,0,2,0,2,29,0,0.0,0.0
31865,A769064,2016,3,2 years,Other,Bat Mix,Brown,3,4,Unknown,1,730,2.000000,0,29-03-2018 17:13,3,2018,2018-03,4,17,1,0 days 01:17:00.000000000,0.0,2 years,29-03-2016 00:00,Unknown,730,2.000000,0,3,2018,2018-03,4,18,1,2,2,0,2,29,0,0.0,0.0
31866,A769065,2017,3,1 year,Other,Bat Mix,Brown,3,4,Unknown,1,365,1.000000,0,29-03-2018 17:19,3,2018,2018-03,4,17,1,0 days 01:09:00.000000000,0.0,1 year,29-03-2017 00:00,Unknown,365,1.000000,0,3,2018,2018-03,4,18,1,2,2,0,2,29,0,0.0,0.0
31867,A769066,2017,5,5 months,Dog,Labrador Retriever Mix,Black/White,3,3,Intact Male,1,150,0.410959,0,29-03-2018 18:19,3,2018,2018-03,4,18,1,1 days 18:18:00.000000000,2.0,10 months,01-05-2017 00:00,Intact Male,300,0.821918,0,3,2018,2018-03,2,12,1,1,1,0,0,29,0,0.0,0.0


In [0]:
train = train.drop(['age_upon_intake'], axis = 1)              # repetitive data
test = test.drop(['age_upon_intake'], axis = 1)

train = train.drop(['age_upon_outcome'], axis = 1)             # repetitive data
test = test.drop(['age_upon_outcome'], axis = 1)

train = train.drop(['breed'], axis = 1)                        # new breed made
test = test.drop(['breed'], axis = 1)

train = train.drop(['animal_type'], axis = 1)                  # new animal_type made
test = test.drop(['animal_type'], axis = 1)

train = train.drop(['sex_upon_intake'], axis = 1)              # gender made, same intake and outcome
test = test.drop(['sex_upon_intake'], axis = 1)

train = train.drop(['count'], axis = 1)                        # remains constant for whole data
test = test.drop(['count'], axis = 1)

train = train.drop(['intake_datetime'], axis = 1)              # all features extracted
test = test.drop(['intake_datetime'], axis = 1)

train = train.drop(['intake_monthyear'], axis = 1)             # no need for month year
test = test.drop(['intake_monthyear'], axis = 1)

train = train.drop(['time_in_shelter'], axis = 1)              # already has days in shelter
test = test.drop(['time_in_shelter'], axis = 1)

train = train.drop(['date_of_birth'], axis = 1)                # day, month, year extracted
test = test.drop(['date_of_birth'], axis = 1)              

train = train.drop(['sex_upon_outcome'], axis = 1)             # gender made, same intake and outcome
test = test.drop(['sex_upon_outcome'], axis = 1)              

train = train.drop(['age_upon_outcome_age_group'], axis = 1)   # difference taken
test = test.drop(['age_upon_outcome_age_group'], axis = 1)

train = train.drop(['outcome_number'], axis = 1)                # same intake and outcome
test = test.drop(['outcome_number'], axis = 1)

train = train.drop(['outcome_monthyear'], axis = 1)              # values extracted, no more needed
test = test.drop(['outcome_monthyear'], axis = 1)

train = train.drop(['color'], axis = 1)                         # converted into numerical categorical
test = test.drop(['color'], axis = 1)

train = train.drop(['age_upon_intake_age_group'], axis = 1)              # converted it into numerical categorical
test = test.drop(['age_upon_intake_age_group'], axis = 1)

train = train.drop(['age_upon_intake_(years)'], axis = 1)              # years in awc taken
test = test.drop(['age_upon_intake_(years)'], axis = 1)

train = train.drop(['age_upon_outcome_(years)'], axis = 1)              # years in awc taken
test = test.drop(['age_upon_outcome_(years)'], axis = 1)

train = train.drop(['dob_year'], axis = 1)              # no more required/ already have age
test = test.drop(['dob_year'], axis = 1)

train = train.drop(['dob_month'], axis = 1)              # no more required/ already have age
test = test.drop(['dob_month'], axis = 1)

train = train.drop(['intake_number'], axis = 1)                # intake number is same as outcome number
test = test.drop(['intake_number'], axis = 1)

In [68]:
train

,animal_id_outcome,intake_condition,intake_type,age_upon_intake_(days),intake_month,intake_year,intake_weekday,intake_hour,time_in_shelter_days,age_upon_outcome_(days),outcome_month,outcome_year,outcome_weekday,outcome_hour,outcome_type,new_animal_type,gender,new_breed,color_new,intake_date,age_group_difference,years_in_awc,months_in_awc
0,A006100,3,3,3650,12,2017,4,14,1.0,3650,12,2017,4,0,Return to Owner,1,1,0,0,7,0,0.0,0.0
1,A006100,3,2,2555,12,2014,0,10,1.0,2555,12,2014,2,16,Return to Owner,1,1,0,0,19,0,0.0,0.0
2,A006100,3,2,2190,3,2014,0,14,1.0,2190,3,2014,2,17,Return to Owner,1,1,0,0,7,0,0.0,0.0
3,A047759,3,1,3650,4,2014,6,15,5.0,3650,4,2014,1,15,Missing,1,1,1,3,2,0,0.0,0.0
4,A134067,2,2,5840,11,2013,2,9,0.0,5840,11,2013,2,11,Return to Owner,1,1,1,0,16,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47798,A723128,3,3,2920,3,2016,3,11,22.0,2920,4,2016,1,0,Rto-Adopt,0,1,0,0,27,0,0.0,0.0
47799,A723129,3,1,365,3,2016,3,11,2.0,365,3,2016,5,19,Adoption,1,1,0,0,27,0,0.0,0.0
47800,A723130,3,3,4015,3,2016,3,11,0.0,4015,3,2016,3,17,Return to Owner,1,0,0,0,27,0,0.0,0.0
47801,A723131,3,3,1825,3,2016,3,11,13.0,1825,4,2016,2,13,Adoption,1,1,2,0,27,0,0.0,0.0


In [69]:
test

,animal_id_outcome,intake_condition,intake_type,age_upon_intake_(days),intake_month,intake_year,intake_weekday,intake_hour,time_in_shelter_days,age_upon_outcome_(days),outcome_month,outcome_year,outcome_weekday,outcome_hour,new_animal_type,gender,new_breed,color_new,intake_date,age_group_difference,years_in_awc,months_in_awc
0,A723133,4,3,1095,3,2016,3,11,5.0,1095,4,2016,0,16,0,0,0,2,27,0,0.0,0.0
1,A723134,4,3,28,3,2016,3,11,34.0,60,4,2016,2,17,0,0,0,2,27,0,0.0,0.0
2,A723135,4,3,28,3,2016,3,11,33.0,60,4,2016,0,0,0,1,0,2,27,0,0.0,0.0
3,A723136,4,3,28,3,2016,3,11,31.0,30,4,2016,6,13,0,0,0,2,27,0,0.0,0.0
4,A723137,4,3,28,3,2016,3,11,37.0,60,5,2016,5,11,0,0,0,2,27,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31864,A769060,3,3,14,3,2018,4,16,0.0,14,3,2018,4,18,0,2,0,2,29,0,0.0,0.0
31865,A769064,3,4,730,3,2018,4,17,0.0,730,3,2018,4,18,2,2,0,2,29,0,0.0,0.0
31866,A769065,3,4,365,3,2018,4,17,0.0,365,3,2018,4,18,2,2,0,2,29,0,0.0,0.0
31867,A769066,3,3,150,3,2018,4,18,2.0,300,3,2018,2,12,1,1,0,0,29,0,0.0,0.0


### Training the Modeling

In [0]:
from sklearn.model_selection import train_test_split as train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , confusion_matrix

In [0]:
y = train['outcome_type']
x = train.drop("outcome_type", axis = 1)

In [0]:
xx = x.drop(['animal_id_outcome'],axis = 1)         # dropping id of animal. It will be added on the testing data

In [73]:
x_train, x_test, y_train, y_test = train_test_split(xx, y, test_size=0.10, random_state=42)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(43021, 21) (43021,)
(4781, 21) (4781,)


In [0]:
# Training model with random forest

In [0]:
rf_clf = RandomForestClassifier(n_estimators=500, random_state=42)

In [76]:
rf_clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [77]:
predictions = rf_clf.predict(x_test)
predictions

array(['Adoption', 'Return to Owner', 'Transfer', ..., 'Adoption',
       'Adoption', 'Adoption'], dtype=object)

In [0]:
accuracy = accuracy_score(y_test, predictions)
print(accuracy*100,"%")

In [0]:
#feature importance

In [0]:
fi = pd.DataFrame({'feature': list(x_train.columns),
                   'importance': rf_clf.feature_importances_}).\
                    sort_values('importance', ascending = False)

In [80]:
fi.head(10)

,feature,importance
7,time_in_shelter_days,0.151198
12,outcome_hour,0.082382
17,intake_date,0.079486
1,intake_type,0.069520
6,intake_hour,0.069043
8,age_upon_outcome_(days),0.068558
2,age_upon_intake_(days),0.066075
9,outcome_month,0.052671
3,intake_month,0.052434
5,intake_weekday,0.051352


In [0]:
#confusion matrics

In [0]:
cm = confusion_matrix(y_test, predictions) 
cm

In [0]:
# Correlation

In [0]:
from scipy.stats import pearsonr
corr, _ = pearsonr(train['age_upon_intake_(days)'], train['age_upon_outcome_(days)'])
corr

In [0]:
corr_matrix = train.corr().abs()
corr_matrix

### Predicting outcome_type on testing data

In [0]:
train_id = train['animal_id_outcome']

In [0]:
test_id = test['animal_id_outcome']

In [0]:
train = train.drop(['animal_id_outcome'], axis = 1)

In [0]:
test = test.drop(['animal_id_outcome'], axis = 1)

In [98]:
train

,intake_condition,intake_type,age_upon_intake_(days),intake_month,intake_year,intake_weekday,intake_hour,time_in_shelter_days,age_upon_outcome_(days),outcome_month,outcome_year,outcome_weekday,outcome_hour,outcome_type,new_animal_type,gender,new_breed,color_new,intake_date,age_group_difference,years_in_awc,months_in_awc
0,3,3,3650,12,2017,4,14,1.0,3650,12,2017,4,0,Return to Owner,1,1,0,0,7,0,0.0,0.0
1,3,2,2555,12,2014,0,10,1.0,2555,12,2014,2,16,Return to Owner,1,1,0,0,19,0,0.0,0.0
2,3,2,2190,3,2014,0,14,1.0,2190,3,2014,2,17,Return to Owner,1,1,0,0,7,0,0.0,0.0
3,3,1,3650,4,2014,6,15,5.0,3650,4,2014,1,15,Missing,1,1,1,3,2,0,0.0,0.0
4,2,2,5840,11,2013,2,9,0.0,5840,11,2013,2,11,Return to Owner,1,1,1,0,16,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47798,3,3,2920,3,2016,3,11,22.0,2920,4,2016,1,0,Rto-Adopt,0,1,0,0,27,0,0.0,0.0
47799,3,1,365,3,2016,3,11,2.0,365,3,2016,5,19,Adoption,1,1,0,0,27,0,0.0,0.0
47800,3,3,4015,3,2016,3,11,0.0,4015,3,2016,3,17,Return to Owner,1,0,0,0,27,0,0.0,0.0
47801,3,3,1825,3,2016,3,11,13.0,1825,4,2016,2,13,Adoption,1,1,2,0,27,0,0.0,0.0


In [0]:
train_label = train['outcome_type']

In [0]:
train_feature = train.drop(['outcome_type'],axis = 1)

In [101]:
train_feature

,intake_condition,intake_type,age_upon_intake_(days),intake_month,intake_year,intake_weekday,intake_hour,time_in_shelter_days,age_upon_outcome_(days),outcome_month,outcome_year,outcome_weekday,outcome_hour,new_animal_type,gender,new_breed,color_new,intake_date,age_group_difference,years_in_awc,months_in_awc
0,3,3,3650,12,2017,4,14,1.0,3650,12,2017,4,0,1,1,0,0,7,0,0.0,0.0
1,3,2,2555,12,2014,0,10,1.0,2555,12,2014,2,16,1,1,0,0,19,0,0.0,0.0
2,3,2,2190,3,2014,0,14,1.0,2190,3,2014,2,17,1,1,0,0,7,0,0.0,0.0
3,3,1,3650,4,2014,6,15,5.0,3650,4,2014,1,15,1,1,1,3,2,0,0.0,0.0
4,2,2,5840,11,2013,2,9,0.0,5840,11,2013,2,11,1,1,1,0,16,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47798,3,3,2920,3,2016,3,11,22.0,2920,4,2016,1,0,0,1,0,0,27,0,0.0,0.0
47799,3,1,365,3,2016,3,11,2.0,365,3,2016,5,19,1,1,0,0,27,0,0.0,0.0
47800,3,3,4015,3,2016,3,11,0.0,4015,3,2016,3,17,1,0,0,0,27,0,0.0,0.0
47801,3,3,1825,3,2016,3,11,13.0,1825,4,2016,2,13,1,1,2,0,27,0,0.0,0.0


In [102]:
test

,intake_condition,intake_type,age_upon_intake_(days),intake_month,intake_year,intake_weekday,intake_hour,time_in_shelter_days,age_upon_outcome_(days),outcome_month,outcome_year,outcome_weekday,outcome_hour,new_animal_type,gender,new_breed,color_new,intake_date,age_group_difference,years_in_awc,months_in_awc
0,4,3,1095,3,2016,3,11,5.0,1095,4,2016,0,16,0,0,0,2,27,0,0.0,0.0
1,4,3,28,3,2016,3,11,34.0,60,4,2016,2,17,0,0,0,2,27,0,0.0,0.0
2,4,3,28,3,2016,3,11,33.0,60,4,2016,0,0,0,1,0,2,27,0,0.0,0.0
3,4,3,28,3,2016,3,11,31.0,30,4,2016,6,13,0,0,0,2,27,0,0.0,0.0
4,4,3,28,3,2016,3,11,37.0,60,5,2016,5,11,0,0,0,2,27,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31864,3,3,14,3,2018,4,16,0.0,14,3,2018,4,18,0,2,0,2,29,0,0.0,0.0
31865,3,4,730,3,2018,4,17,0.0,730,3,2018,4,18,2,2,0,2,29,0,0.0,0.0
31866,3,4,365,3,2018,4,17,0.0,365,3,2018,4,18,2,2,0,2,29,0,0.0,0.0
31867,3,3,150,3,2018,4,18,2.0,300,3,2018,2,12,1,1,0,0,29,0,0.0,0.0


In [104]:
classifier = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state = 42)
classifier.fit(train_feature, train_label)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [105]:
predictions = classifier.predict(test)
predictions

array(['Transfer', 'Adoption', 'Adoption', ..., 'Euthanasia',
       'Return to Owner', 'Return to Owner'], dtype=object)

In [0]:
output = pd.DataFrame({'animal_id_outcome': test_id, 
                       'outcome_type' : predictions})

In [107]:
output

,animal_id_outcome,outcome_type
0,A723133,Transfer
1,A723134,Adoption
2,A723135,Adoption
3,A723136,Adoption
4,A723137,Adoption
...,...,...
31864,A769060,Transfer
31865,A769064,Euthanasia
31866,A769065,Euthanasia
31867,A769066,Return to Owner


In [0]:
output.to_csv('predictions.csv')